## Neo4j Import of GraphRAG Result Parquet files
This notebook imports the results of the GraphRAG indexing process into the Neo4j Graph database for further processing, analysis or visualization.

### How does it work?
The notebook loads the parquet files from the output folder of your indexing process and loads them into Pandas dataframes. It then uses a batching approach to send a slice of the data into Neo4j to create nodes and relationships and add relevant properties. The id-arrays on most entities are turned into relationships.

All operations use `MERGE`, so they are idempotent, and you can run the script multiple times.

If you need to clean out the database, you can run the following statement
```
MATCH (n)
CALL { WITH n DETACH DELETE n } IN TRANSACTIONS OF 25000 ROWS;
```

In [2]:
GRAPHRAG_FOLDER="ragtest/output"

In [1]:
import pandas as pd
from neo4j import GraphDatabase
import time

In [6]:
from dotenv import load_dotenv
import os

# Specify the exact path to your .env file
ENV_PATH = r"./ragtest/.env"
load_dotenv(ENV_PATH)
NEO4J_URI="neo4j+s://c55a8a58.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD=os.getenv("NEO4J_PASSWORD") 
NEO4J_DATABASE="neo4j"
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [7]:
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.
    Parameters: statement is the Cypher query to execute, df is the dataframe to import, and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0,total, batch_size):
        batch = df.iloc[start: min(start+batch_size,total)]
        result = driver.execute_query("UNWIND $rows AS value " + statement, 
                                      rows=batch.to_dict('records'),
                                      database_=NEO4J_DATABASE)
        print(result.summary.counters)
    print(f'{total} rows in { time.time() - start_s} s.')    
    return total

### Indexes and Constraints
Indexes in Neo4j are only used to find the starting points for graph queries, e.g. quickly finding two nodes to connect. Constraints exist to avoid duplicates, we create them mostly on id's of Entity types.

We use some Types as markers with two underscores before and after to distinguish them from the actual entity types.

The default relationship type here is `RELATED` but we could also infer a real relationship-type from the description or the types of the start and end-nodes.

* `__Entity__`
* `__Document__`
* `__Chunk__`
* `__Community__`
* `__Covariate__`

In [8]:
# create constraints, idempotent operation

statements = """
create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique;
create constraint document_id if not exists for (d:__Document__) require d.id is unique;
create constraint entity_id if not exists for (c:__Community__) require c.community is unique;
create constraint entity_id if not exists for (e:__Entity__) require e.id is unique;
create constraint entity_title if not exists for (e:__Entity__) require e.name is unique;
create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique;
create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique;
""".split(";")

for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique

create constraint document_id if not exists for (d:__Document__) require d.id is unique

create constraint entity_id if not exists for (c:__Community__) require c.community is unique

create constraint entity_id if not exists for (e:__Entity__) require e.id is unique

create constraint entity_title if not exists for (e:__Entity__) require e.name is unique

create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique

create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique


## Import Process
### Importing the Documents
We're loading the parquet file for the documents and create nodes with their ids and add the title property. We don't need to store text_unit_ids as we can create the relationships and the text content is also contained in the chunks.

In [13]:
doc_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_documents.parquet', columns=["id", "title"])
doc_df.head(2)

,id,title
0,d4f8f5124d9130835e57f26ac7e28507,Behind the Tech_Episode 30_Collier_Bloomberg_T...
1,4d09f0916d76792fbb7ed5487cd789a5,Behind the Tech_Episode 35_StevenBathiche_Tran...


In [14]:
# import documents
statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.title}
"""

batched_import(statement, doc_df)

{'_contains_updates': True, 'labels_added': 9, 'nodes_created': 9, 'properties_set': 19}
10 rows in 0.3578355312347412 s.


10

### Loading Text Units
We load the text units, create a node per id and set the text and number of tokens. Then we connect them to the documents that we created before.

In [15]:
text_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_text_units.parquet',
                          columns=["id","text","n_tokens","document_ids"])
text_df.head(2)

,id,text,n_tokens,document_ids
0,d860068315890bae779295efc52a29a6,\nMAE JEMISON: 100 Year Starship is about maki...,1200.0,[00ff4fca30e291bc1b3fe51ef4563cde]
1,9e859295e56ccc9aa92432531da89338,"\n\nBecause if you think about, as you’re sit...",1200.0,[00ff4fca30e291bc1b3fe51ef4563cde]


In [16]:
statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens}
WITH c, value
UNWIND value.document_ids AS document
MATCH (d:__Document__ {id:document})
MERGE (c)-[:PART_OF]->(d)
"""

batched_import(statement, text_df)

{'_contains_updates': True, 'labels_added': 64, 'relationships_created': 64, 'nodes_created': 64, 'properties_set': 192}
64 rows in 0.6005172729492188 s.


64

### Loading Nodes
For the nodes we store id, name, description, embedding (if available), human readable id.

In [17]:
entity_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_entities.parquet',
                            columns=["name","type","description","human_readable_id","id","description_embedding","text_unit_ids"])
entity_df.head(2)

,name,type,description,human_readable_id,id,description_embedding,text_unit_ids
0,MAE JEMISON,PERSON,Mae Jemison is a distinguished former NASA ast...,0,0bea32879ccd47a8a888678673ee3028,"[0.06652918457984924, -0.025584489107131958, 0...","[0900d8a693b4d81016ece7a5084ca4db, 123b198d891..."
1,KEVIN SCOTT,PERSON,Kevin Scott is a prominent computer scientist ...,1,59a02b4714ad452d9845be266aab4a78,"[0.0350932851433754, -0.04698536545038223, -0....","[0900d8a693b4d81016ece7a5084ca4db, 09510a1363e..."


In [18]:
entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET e += value {.human_readable_id, .description, name:replace(value.name,'"','')}
WITH e, value
CALL db.create.setNodeVectorProperty(e, "description_embedding", value.description_embedding)
CALL apoc.create.addLabels(e, case when coalesce(value.type,"") = "" then [] else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) yield node
UNWIND value.text_unit_ids AS text_unit
MATCH (c:__Chunk__ {id:text_unit})
MERGE (c)-[:HAS_ENTITY]->(e)
"""

batched_import(entity_statement, entity_df)

{'_contains_updates': True, 'labels_added': 356, 'relationships_created': 593, 'nodes_created': 356, 'properties_set': 1424}
356 rows in 2.044311285018921 s.


356

### Import Relationships
For the relationships we find the source and target node by name, using the base `__Entity__` type. After creating the RELATED relationships, we set the description as attribute.

In [19]:
rel_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_relationships.parquet',
                         columns=["source","target","id","rank","weight","human_readable_id","description","text_unit_ids"])
rel_df.head(2)

,source,target,id,rank,weight,human_readable_id,description,text_unit_ids
0,MAE JEMISON,CHICAGO PUBLIC SCHOOLS,9ccbda9e69d14dfab301d476668984d2,37,8.0,0,"Mae Jemison, an accomplished astronaut and phy...","[749b7537a1839bf99075a0852496d717, 9e859295e56..."
1,MAE JEMISON,STANFORD,39c455d424d04480a8ead590cc081cb2,38,8.0,1,Mae Jemison attended Stanford University at a ...,[9e859295e56ccc9aa92432531da89338]


In [20]:
rel_statement = """
    MATCH (source:__Entity__ {name:replace(value.source,'"','')})
    MATCH (target:__Entity__ {name:replace(value.target,'"','')})
    // not necessary to merge on id as there is only one relationship per pair
    MERGE (source)-[rel:RELATED {id: value.id}]->(target)
    SET rel += value {.rank, .weight, .human_readable_id, .description, .text_unit_ids}
    RETURN count(*) as createdRels
"""

batched_import(rel_statement, rel_df)

{'_contains_updates': True, 'relationships_created': 431, 'properties_set': 2586}
431 rows in 0.7791140079498291 s.


431

### Importing Communities
For communities we import their id, title, level. We connect the `__Community__` nodes to the start and end nodes of the relationships they refer to.

Connecting them to the chunks they orignate from is optional, as the entites are already connected to the chunks.

In [21]:
community_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_communities.parquet', 
                     columns=["id","level","title","text_unit_ids","relationship_ids"])

community_df.head(2)

,id,level,title,text_unit_ids,relationship_ids
0,3,0,Community 3,"[0900d8a693b4d81016ece7a5084ca4db,123b198d891b...","[9ccbda9e69d14dfab301d476668984d2, 39c455d424d..."
1,1,0,Community 1,"[0900d8a693b4d81016ece7a5084ca4db,09510a1363ec...","[3539650af2204e6db318f7b6079697e2, dfd587694e6..."


In [22]:
statement = """
MERGE (c:__Community__ {community:value.id})
SET c += value {.level, .title}
/*
UNWIND value.text_unit_ids as text_unit_id
MATCH (t:__Chunk__ {id:text_unit_id})
MERGE (c)-[:HAS_CHUNK]->(t)
WITH distinct c, value
*/
WITH *
UNWIND value.relationship_ids as rel_id
MATCH (start:__Entity__)-[:RELATED {id:rel_id}]->(end:__Entity__)
MERGE (start)-[:IN_COMMUNITY]->(c)
MERGE (end)-[:IN_COMMUNITY]->(c)
RETURN count(distinct c) as createdCommunities
"""

batched_import(statement, community_df)

{'_contains_updates': True, 'labels_added': 61, 'relationships_created': 1299, 'nodes_created': 61, 'properties_set': 183}
61 rows in 0.7351629734039307 s.


61

### Importing Community Reports
Fo the community reports we create nodes for each communitiy set the id, community, level, title, summary, rank, and rank_explanation and connect them to the entities they are about. For the findings we create the findings in context of the communities.

In [23]:
community_report_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_community_reports.parquet',
                               columns=["id","community","level","title","summary", "findings","rank","rank_explanation","full_content"])
community_report_df.head(2)

,id,community,level,title,summary,findings,rank,rank_explanation,full_content
0,ac1e72f1-0238-4844-89c7-3a367cd358f4,59,4,Kevin Scott and the Future of Technology,"The community centers around Kevin Scott, the ...",[{'explanation': 'Kevin Scott serves as the Ch...,8.5,The impact severity rating is high due to the ...,# Kevin Scott and the Future of Technology\n\n...
1,b53a5f14-6ceb-44bf-8888-be71f3e51982,60,4,Technology and Its Impact on Society,The community focuses on the intersection of t...,[{'explanation': 'Kevin Scott is a prominent f...,7.5,The impact severity rating is high due to the ...,# Technology and Its Impact on Society\n\nThe ...


In [24]:
# import communities
community_statement = """
MERGE (c:__Community__ {community:value.community})
SET c += value {.level, .title, .rank, .rank_explanation, .full_content, .summary}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id:finding_idx})
SET f += finding
"""
batched_import(community_statement, community_report_df)

{'_contains_updates': True, 'labels_added': 304, 'relationships_created': 304, 'nodes_created': 304, 'properties_set': 1278}
61 rows in 0.36124753952026367 s.


61

### Importing Covariates
Covariates are for instance claims on entities, we connect them to the chunks where they originate from.

**By default, covariates are not included in the output, so the file might not exists in your output if you didn't set the configuration to extract claims**

In [25]:
"""
# cov_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_covariates.parquet')
# cov_df.head(2)
"""

"\n# cov_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_covariates.parquet')\n# cov_df.head(2)\n"

In [26]:
cov_statement = """
MERGE (c:__Covariate__ {id:value.id})
SET c += apoc.map.clean(value, ["text_unit_id", "document_ids", "n_tokens"], [NULL, ""])
WITH c, value
MATCH (ch:__Chunk__ {id: value.text_unit_id})
MERGE (ch)-[:HAS_COVARIATE]->(c)
"""
# batched_import(cov_statement, cov_df)